In [ ]:
from YouTubeDownload import YouTubeDownload
from MusicSeparation import MusicSeparation
from pytubefix import YouTube
import pytubefix
import os

url = input()

def download_youtube_audio(video_url, mode = "audio"):
    if mode == "audio" :
        try:
            yt = YouTube(video_url)
            title = YouTubeDownload.sanitize_filename(yt.title)
            audio_stream = yt.streams.get_audio_only()

            os.makedirs("download", exist_ok=True)
            os.path.join("download", f"{title}.m4a")
            m4a_path = audio_stream.download(output_path="download", filename=f"{title}.m4a")

            return f'"{m4a_path}"'
        except Exception as e:
            return f"錯誤：{e}"
    elif mode == "playlist":
        for audio in pytubefix.Playlist(video_url):
            print(audio)
            temp = []
            try:
                yt = YouTube(audio)
                title = YouTubeDownload.sanitize_filename(yt.title)
                audio_stream = yt.streams.get_audio_only()

                os.makedirs("download", exist_ok=True)
                os.path.join("download", f"{title}.m4a")
                temp.append(audio_stream.download(output_path="download", filename=f"{title}.m4a"))
                
            except Exception as e:
                temp.append(f"錯誤：{e}")
        return temp
#download_youtube_audio(url, mode="playlist")
audio = YouTubeDownload.download_youtube_audio(url, mode="playlist")
audio


In [ ]:
temp = []
for str1 in audio:
    temp.append(MusicSeparation.run_separation(str1))
temp

In [1]:
import os
import csv
import glob
import json
import shutil
import tempfile
import subprocess
import sys
from pydub import AudioSegment

# (align_song_aeneas 函式不變，請保留您原有的版本)
# ...
def align_song_aeneas(audio_path, output_dir):
 """
 使用 Aeneas 對齊歌曲 + 歌詞，並使用 Pydub 切割成一句一句音檔 + CSV
 (最終版：使用 subprocess 呼叫命令列，繞過所有 Bug)
 """
 os.makedirs(output_dir, exist_ok=True)
 base_name = os.path.splitext(os.path.basename(audio_path))[0]

 # 1. 嘗試找同名 txt 歌詞檔
 lyrics_txt = os.path.join(os.path.dirname(audio_path), f"{base_name}.txt")
 
 if not os.path.exists(lyrics_txt):
  print(f"⚠️ 找不到歌詞檔 {lyrics_txt}，Aeneas 將無法對齊此歌曲。")
  return None
 
 # === AENEAS HARDCODED WORKAROUND ===
 HARDCODED_TEMP_DIR = r"C:\aeneas_temp_workdir"
 os.makedirs(HARDCODED_TEMP_DIR, exist_ok=True)
 
 temp_audio_path = os.path.join(HARDCODED_TEMP_DIR, "temp_audio.wav")
 temp_text_path = os.path.join(HARDCODED_TEMP_DIR, "temp_lyrics.txt")
 
 # *真實* 的 JSON 輸出路徑
 json_path = os.path.join(output_dir, "aligned.json")
 
 try:
  # 3. 複製/轉換檔案到臨時位置
  try:
   audio_for_temp = AudioSegment.from_file(audio_path)
   audio_for_temp.export(temp_audio_path, format="wav")
  except Exception as e:
   print(f"❌ 讀取/複製音訊檔 {base_name} 時 (Pydub) 發生錯誤: {e}")
   return None
   
  shutil.copy(lyrics_txt, temp_text_path)

  # 4. 建立 Aeneas *命令列*
  # 我們使用 sys.executable 來確保我們用的是當前 Python 環境
  command = [
   sys.executable, # 'C:\Program Files\Python39\python.exe'
   "-m", "aeneas.tools.execute_task",
   temp_audio_path,
   temp_text_path,
   "task_language=cmn|is_text_type=plain|os_task_file_format=json",
   os.path.abspath(json_path) # 確保 Aeneas 寫入正確位置
  ]

  print(f"🎤 Working on: {base_name}")
  print(f"(Aeneas 正在對齊 {os.path.basename(audio_path)} ...)")

  # 5. 執行 Aeneas 命令列
  # 我們不再使用 aeneas 函式庫，而是直接呼叫我們手動測試成功的命令
  result = subprocess.run(
   command, 
   capture_output=True, 
   text=True, 
   encoding="utf-8"
  )

  # 6. 檢查命令是否成功
  if result.returncode != 0:
   print(f"❌ Aeneas 對齊失敗 {base_name}。")
   print("--- Aeneas Error Output ---")
   print(result.stdout)
   print(result.stderr)
   print("---------------------------")
   return None

 except Exception as e:
  print(f"❌ Aeneas 發生未預期的錯誤 (subprocess): {e}")
  return None
  
 finally:
  # 7. 關鍵：刪除臨時檔案
  if os.path.exists(temp_audio_path):
   os.remove(temp_audio_path)
  if os.path.exists(temp_text_path):
   os.remove(temp_text_path)
 
 # === WORKAROUND 結束 ===
 
 print(f"(Aeneas 對齊完成，正在切割音檔...)")

 # 8. 讀取 Aeneas 輸出的 JSON，並使用 Pydub 切割
 try:
  audio = AudioSegment.from_file(audio_path)
  
  with open(json_path, 'r', encoding='utf-8') as f:
   align_data = json.load(f)

  csv_path = os.path.join(output_dir, "metadata.csv")
  with open(csv_path, "w", newline="", encoding="utf-8") as f:
   writer = csv.writer(f)
   writer.writerow(["file", "text"]) 

   fragments = align_data.get("fragments")
   if not fragments:
    print(f"⚠️ 對齊的 JSON {json_path} 中沒有找到 'fragments'。")
    return None

   for i, fragment in enumerate(fragments):
    text = fragment.get("lines", [""])[0].strip()
    if not text:
     continue 

    begin_ms = float(fragment["begin"]) * 1000
    end_ms = float(fragment["end"]) * 1000

    segment = audio[begin_ms:end_ms]

    out_wav = os.path.join(output_dir, f"line_{i:04d}.wav")
    segment.export(
     out_wav, 
     format="wav", 
     parameters=["-ar", "16000", "-ac", "1"]
    )
    
    writer.writerow([out_wav, text])

  print(f"(切割完成，已儲存至 {output_dir})")
  return csv_path

 except Exception as e:
  print(f"❌ Pydub 切割或讀取 JSON 時發生錯誤: {e}")
  return None


# === 變更開始 ===

# 1. 為了符合您的範例 "processed_songs"，我們修改函式的預設輸出資料夾
def process_folder_aeneas(input_dir, output_root="processed_songs", merged_csv="metadata_all.csv"):
 """
 批次處理資料夾內所有歌曲，用 Aeneas + Pydub 對齊
 """
 os.makedirs(output_root, exist_ok=True)
 all_rows = []

 audio_files = []
 for ext in ("*.wav", "*.mp3", "*.flac"):
  audio_files.extend(glob.glob(os.path.join(input_dir, ext)))

 if not audio_files:
  print(f"⚠️ 在 {input_dir} 中找不到任何音訊檔案。")
  return

 print(f"總共找到 {len(audio_files)} 個音訊檔案，開始處理...")

 for audio_path in audio_files:
  base_name = os.path.splitext(os.path.basename(audio_path))[0]
  song_output = os.path.join(output_root, base_name)
  
  csv_path = align_song_aeneas(audio_path, song_output)

  if csv_path:
   with open(csv_path, "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    next(reader) # 跳過標頭
    for row in reader:
     # --- 修正開始 ---
     # 我們不再使用 relpath 和 replace
     # row[0] (例如 'processed_songs/song1/line_0001.wav')
     # 已經是我們想要的格式 (包含根目錄)
     # 
     # (原有的 relpath 修正區塊已被移除)
     # --- 修正結束 ---
     
     all_rows.append(row)
   
 merged_path = os.path.join(output_root, merged_csv)
 if not all_rows:
  print("\n❌ 所有歌曲都處理失敗，沒有產生任何 metadata。")
  return
  
 with open(merged_path, "w", newline="", encoding="utf-8") as f:
  writer = csv.writer(f)
  writer.writerow(["file", "text"])
  writer.writerows(all_rows)

 print(f"\n✅ 所有歌曲已處理完成！")
 print(f"👉 輸出資料夾: {output_root}")
 print(f"👉 合併 metadata: {merged_path} (共 {len(all_rows)} 筆資料)")


# --- 如何使用 ---
if __name__ == "__main__":
 # 假設您的歌曲和 .txt 歌詞檔都放在 "my_songs" 資料夾
 INPUT_FOLDER = "song_dataset" 
 
 # 2. 為了符合您的範例，我們修改這裡的輸出資料夾名稱
 OUTPUT_FOLDER = "processed_songs"

 process_folder_aeneas(INPUT_FOLDER, OUTPUT_FOLDER)

# === 變更結束 ===

總共找到 58 個音訊檔案，開始處理...
🎤 Working on: Dawen 王大文 - 練習愛情 ft. Kimberley 陳芳語 _Let's Work It Out_ (Official MV)-vocals
(Aeneas 正在對齊 Dawen 王大文 - 練習愛情 ft. Kimberley 陳芳語 _Let's Work It Out_ (Official MV)-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs\Dawen 王大文 - 練習愛情 ft. Kimberley 陳芳語 _Let's Work It Out_ (Official MV)-vocals)
🎤 Working on: F.I.R. 飛兒樂團 - 我們的愛 (official官方完整版MV)-vocals
(Aeneas 正在對齊 F.I.R. 飛兒樂團 - 我們的愛 (official官方完整版MV)-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs\F.I.R. 飛兒樂團 - 我們的愛 (official官方完整版MV)-vocals)
🎤 Working on: F.I.R. 飛兒樂團 - 月牙灣 Crescent Bay (official 官方完整版MV)-vocals
(Aeneas 正在對齊 F.I.R. 飛兒樂團 - 月牙灣 Crescent Bay (official 官方完整版MV)-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs\F.I.R. 飛兒樂團 - 月牙灣 Crescent Bay (official 官方完整版MV)-vocals)
🎤 Working on: First Day-vocals
(Aeneas 正在對齊 First Day-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs\First Day-vocals)
🎤 Working on: Khalil Fong (方大同) - 愛愛愛 Offici

## 正確率評分

In [ ]:
import Levenshtein
import difflib

#正規化函數，把 "妳"、"祢" 視為 "你"，再視為在
def normalize_text(text: str) -> str:
    mapping = {
        "妳": "你",
        "祢": "你",
        "再": "在",
    }
    return "".join(mapping.get(ch, ch) for ch in text)

def calc_accuracy_lyrics(gt_file, test_file, report_file="compare_report.txt"):
    #讀取檔案
    with open(gt_file, "r", encoding="utf-8") as f:
        gt_text = f.read()
    with open(test_file, "r", encoding="utf-8") as f:
        test_text = f.read()

    #移除空白與換行，並正規化
    gt_text = normalize_text(gt_text.replace(" ", "").replace("\n", ""))
    test_text = normalize_text(test_text.replace(" ", "").replace("\n", ""))

    #計算編輯距離
    distance = Levenshtein.distance(gt_text, test_text)
    max_len = max(len(gt_text), len(test_text))
    accuracy = (max_len - distance) / max_len if max_len > 0 else 0

    #用difflib做逐字比對
    diff = difflib.SequenceMatcher(None, gt_text, test_text)
    report_parts = []
    for tag, i1, i2, j1, j2 in diff.get_opcodes():
        if tag == "equal":
            report_parts.append(gt_text[i1:i2])  
        elif tag == "replace":
            for a, b in zip(gt_text[i1:i2], test_text[j1:j2]):
                report_parts.append(f"【預測:{b}/正確:{a}】")
            if (i2 - i1) > (j2 - j1):
                for a in gt_text[j2 - j1 + i1:i2]:
                    report_parts.append(f"【缺少:{a}】")
            elif (j2 - j1) > (i2 - i1):
                for b in test_text[i2 - i1 + j1:j2]:
                    report_parts.append(f"【多餘:{b}】")
        elif tag == "delete":
            for a in gt_text[i1:i2]:
                report_parts.append(f"【缺少:{a}】")
        elif tag == "insert":
            for b in test_text[j1:j2]:
                report_parts.append(f"【多餘:{b}】")
        report_parts.append('\n')

    result_text = []
    result_text.append(f"總字數: {max_len}")
    result_text.append(f"最少編輯次數: {distance}")
    result_text.append(f"正確率: {accuracy*100:.2f}%")
    result_text.append("\n=== 詳細比對結果===")
    result_text.append("".join(report_parts))

    print("\n".join(result_text))

    with open(report_file, "w", encoding="utf-8") as f:
        f.write("\n".join(result_text))
    print(f"\n比對結果已輸出至 {report_file}")

In [ ]:
#呼叫方法(正確歌詞,測試歌詞)
calc_accuracy_lyrics("青花瓷歌詞.txt", "OutputFile.txt")